In [1]:
# run this script with the SlicerJupyter extension (3D Slicer)
# instructions on how to install the extension and run scripts are available here:
# https://github.com/Slicer/SlicerJupyter

In [2]:

import os
import pandas as pd
from tqdm import tqdm
import numpy as np

from glob import glob
import imageio
import time

import slicer
from slicer.util import array, getNode, loadVolume, saveNode, selectModule, VTKObservationMixin
import vtk

In [3]:
# define a list of paths to the folders containing the masked images
defaced_dirs = glob(os.path.join('./data/example_output/defaced_images', "*"))

# define where the GIFs will be saved
out_root = './data/example_output/defaced_images_3d_visualization'

In [4]:

def showTransparentRendering(volumeNode, maxOpacity=1, gradientThreshold=30.0):
  """Make constant regions transparent and the entire volume somewhat transparent
  :param maxOpacity: lower value makes the volume more transparent overall
    (value is between 0.0 and 1.0)
  :param gradientThreshold: regions that has gradient value below this threshold will be made transparent
    (minimum value is 0.0, higher values make more tissues transparent, starting with soft tissues)
  """
  # Get/create volume rendering display node
  volRenLogic = slicer.modules.volumerendering.logic()
  displayNode = volRenLogic.GetFirstVolumeRenderingDisplayNode(volumeNode)
  if not displayNode:
    displayNode = volRenLogic.CreateDefaultVolumeRenderingNodes(volumeNode)
  # Set up gradient vs opacity transfer function
  gradientOpacityTransferFunction = displayNode.GetVolumePropertyNode().GetVolumeProperty().GetGradientOpacity()
  gradientOpacityTransferFunction.RemoveAllPoints()
  gradientOpacityTransferFunction.AddPoint(0, 0.0)
  gradientOpacityTransferFunction.AddPoint(gradientThreshold-1, 0.0)
  gradientOpacityTransferFunction.AddPoint(gradientThreshold+1, maxOpacity)
  # Show volume rendering
  displayNode.SetVisibility(True)

def showVolumeRendering(volumeNode):
    print("Show volume rendering of node " + volumeNode.GetName())
    volRenLogic = slicer.modules.volumerendering.logic()
    displayNode = volRenLogic.CreateDefaultVolumeRenderingNodes(volumeNode)
    displayNode.SetVisibility(True)
    scalarRange = volumeNode.GetImageData().GetScalarRange()
    displayNode.GetVolumePropertyNode().Copy(volRenLogic.GetPresetByName("MR-Default"))
    
def showVolumeRenderingMIP(volumeNode, useSliceViewColors=True):
  """Render volume using maximum intensity projection
  :param useSliceViewColors: use the same colors as in slice views.
  """
  # Get/create volume rendering display node
  volRenLogic = slicer.modules.volumerendering.logic()
  displayNode = volRenLogic.GetFirstVolumeRenderingDisplayNode(volumeNode)
  if not displayNode:
    displayNode = volRenLogic.CreateDefaultVolumeRenderingNodes(volumeNode)
  # Choose MIP volume rendering preset
  if useSliceViewColors:
    volRenLogic.CopyDisplayToVolumeRenderingDisplayNode(displayNode)
  else:
    scalarRange = volumeNode.GetImageData().GetScalarRange()
    if scalarRange[1]-scalarRange[0] < 1500:
      # Small dynamic range, probably MRI
      displayNode.GetVolumePropertyNode().Copy(volRenLogic.GetPresetByName("MR-MIP"))
    else:
      # Larger dynamic range, probably CT
      displayNode.GetVolumePropertyNode().Copy(volRenLogic.GetPresetByName("CT-MIP"))
  # Switch views to MIP mode
  for viewNode in slicer.util.getNodesByClass("vtkMRMLViewNode"):
    viewNode.SetRaycastTechnique(slicer.vtkMRMLViewNode.MaximumIntensityProjection)
  # Show volume rendering
  displayNode.SetVisibility(True)
  

problems = []
for folder in tqdm(defaced_dirs):
        print(f"processing {folder=}")
        outputimages = os.path.join(out_root, os.path.basename(folder))

        os.makedirs(outputimages, exist_ok=True)

        imgs =  [k for k in os.listdir(folder) if '_masked.nii.gz' in k]

        for img in imgs:
            loadedVolumeNode = slicer.util.loadVolume(os.path.join(folder,img))
            
            nodeName = loadedVolumeNode.GetName()
            voxelArray = array(nodeName) # get voxels as numpy array
            thresholdValue = np.quantile(voxelArray, 0.2)
            voxelArray[voxelArray < thresholdValue] = 0 # modify voxel values
            getNode(nodeName).Modified() # at the end of all processing, notify Slicer that the image modification is completed
            #showVolumeRenderingMIP(loadedVolumeNode)

            if folder=='Preop-MR':
                showVolumeRendering(loadedVolumeNode)
            else:
                showTransparentRendering(loadedVolumeNode, 1, 0.0)
            
            view = slicer.app.layoutManager().threeDWidget(0).threeDView()
            view.mrmlViewNode().SetBackgroundColor(0,0,0)
            view.mrmlViewNode().SetBackgroundColor2(0,0,0)

            layoutManager = slicer.app.layoutManager()
            threeDWidget = layoutManager.threeDWidget(0)
            threeDView = threeDWidget.threeDView()
            threeDView.resetFocalPoint()
            
            slicer.util.forceRenderAllViews()
            slicer.app.processEvents()

            list_imgs = []     
            for i in range(76):
                layoutManager = slicer.app.layoutManager()
                threeDWidget = layoutManager.threeDWidget(0)
                threeDView = threeDWidget.threeDView()
                threeDView.yaw()

                
                if i%5==0:
                    renderWindow = view.renderWindow()
                    renderWindow.SetAlphaBitPlanes(1)
                    wti = vtk.vtkWindowToImageFilter()
                    wti.SetInputBufferTypeToRGBA()
                    wti.SetInput(renderWindow)
                    writer = vtk.vtkPNGWriter()
                    path_img_view = os.path.join(outputimages,img.replace('nii.gz',f'{i:03}.png'))
                    list_imgs.append(path_img_view)
                    writer.SetFileName(path_img_view)
                    writer.SetInputConnection(wti.GetOutputPort())
                    writer.Write()
                    
                
            images = []
            for filename in list_imgs[1:]:
                images.append(imageio.imread(filename))
            imageio.mimsave(os.path.join(outputimages,img.replace('.nii.gz','.gif')), images)

            for filename in list_imgs:
                os.remove(filename)
            
            slicer.mrmlScene.RemoveNode(loadedVolumeNode)
            slicer.mrmlScene.Clear(0)

  0%|          | 0/2 [00:00<?, ?it/s]

processing folder='./data/example_output/defaced_images/IXI002-Guys-0828-T1'


/tmp/xpython_577996/3422655470.py:117: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
 50%|#####     | 1/2 [00:03<00:03,  3.86s/it]

processing folder='./data/example_output/defaced_images/IXI002-Guys-0828-T2'


100%|##########| 2/2 [00:07<00:00,  3.65s/it]
